In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
sys.path.append("../..")
from data import utils
from src.configuration import load_config
import matplotlib.pyplot as plt

In [ ]:
dir_path = Path("/home/beomyeol/Workspace/ibm/faro/results/faro-us-south/resnet34/top9_twitter_1_1600_avgproc_min_int5m_reduced_6hr_augmented/32_cpus/aiad")
metric_path = dir_path / "metrics.pkl.gz"
df = pd.read_pickle(metric_path)
config_path = dir_path / "config.yaml"
config = load_config(config_path)

In [ ]:
if "autoscale_config" in config:
    target_metric = config["autoscale_config"]["target_metric"]
else:
    target_metric = config["policies"][0]["autoscale_config"]["target_metric"]
print(target_metric)

In [ ]:
df.succeeded.value_counts()

In [ ]:
df["utility"] = (target_metric / df.latency_ms).clip(lower=None, upper=1.0)
df.loc[~df["succeeded"], "utility"] = 0
df["slo"] = ((df.latency_ms <= target_metric) & df.succeeded)

In [ ]:
(df.utility == 0).value_counts()

In [ ]:
df.utility.hist()

In [ ]:
avg_utils = utils.calculate_aggregate(df.utility, df.arrival_ts, 60)
slo_rates = utils.calculate_aggregate(df.slo, df.arrival_ts, 60)
pd.Series(avg_utils).plot(label="utility")
pd.Series(slo_rates).plot(label="slo rate")
plt.legend()

In [ ]:
# correlation plot
import seaborn as sns
import scipy
import matplotlib

matplotlib.rcParams.update({'font.size': 13})

x = "slo_rate"
y = "avg_utility"

def annotate(data, **kws):
    r, p = scipy.stats.pearsonr(data[x], data[y])
    ax = plt.gca()
    ax.text(.05, .8, 'r={:.2f}, p={:.2g}'.format(r, p),
            transform=ax.transAxes)

slo_rate_utility_df = pd.DataFrame({
    x: slo_rates,
    y: avg_utils,
})
# print(len(slo_rate_utility_df))
# slo_rate_utility_df[x] #/= 100
# a, b = np.polyfit(slo_rate_utility_df[x], slo_rate_utility_df[y], deg=1)
# print(a, b)

# sns.scatterplot(x=x, y=y, data=slo_rate_utility_df)
# g = sns.lmplot(x=x, y=y, data=slo_rate_utility_df)
# g.map_dataframe(annotate)

ax = slo_rate_utility_df.plot.scatter(x=x, y=y)
x = np.linspace(0, 1, 10)
# ax.plot(x, a * x + b, color="k")
ax.plot(x, x, color="r")
# ax.text(0.05, 0.9, f"r={slo_rate_utility_df.corr().iloc[1, 0]:.2f}")
ax.set_xlabel("SLO satisfaction rate")
# ax.set_ylabel("Average Utility")
ax.set_ylabel("Utility")
plt.savefig("images/correlation.eps", bbox_inches="tight")